In [616]:
# libs
import os
import pandas as pd
# show all the columns in pandas df
pd.set_option('display.max_columns', None)
import requests
from dotenv import load_dotenv #pip3 install python-dotenv
# load secrets
load_dotenv()
import math
import json
from IPython.display import JSON


In [617]:
# iex cloud - api data request
iex_key = os.getenv("IEX_API_KEY")
url = 'https://cloud.iexapis.com/stable/ref-data/symbols?token=' + iex_key
response = requests.get(url)
data = response.json()
symbols_df = pd.DataFrame(data)
# print 
symbols_df.head()

,symbol,exchange,exchangeSuffix,exchangeName,exchangeSegment,exchangeSegmentName,name,date,type,iexId,region,currency,isEnabled,figi,cik,lei
0,A,XNYS,,New York Stock Exchange Inc,XNYS,New York Stock Exchange Inc,Agilent Technologies Inc.,2024-02-21,cs,IEX_46574843354B2D52,US,USD,True,BBG000C2V3D6,0001090872,QUIX8Y7A2WP0XRMW7G29
1,AA,XNYS,,New York Stock Exchange Inc,XNYS,New York Stock Exchange Inc,Alcoa Corp,2024-02-21,cs,IEX_4238333734532D52,US,USD,True,BBG00B3T3HD3,0001675149,549300T12EZ1F6PWWU29
2,AAA,ARCX,,Nyse Arca,ARCX,Nyse Arca,Investment Managers Series Trust II - AXS Firs...,2024-02-21,et,IEX_5030314338392D52,US,USD,True,BBG01B0JRCS6,0001587982,549300SU7ER9OFETRU41
3,AAAU,BATS,,Cboe Bzx U S Equities Exchange,BATS,Cboe Bzx U S Equities Exchange,Goldman Sachs Physical Gold ETF Trust - Goldma...,2024-02-21,et,IEX_474B433136332D52,US,USD,True,BBG00LPXX872,0001708646,None
4,AACG,XNAS,,Nasdaq All Markets,XNMS,Nasdaq Nms Global Market,ATA Creativity Global - ADR,2024-02-21,ad,IEX_44595A4C53392D52,US,USD,True,BBG000V2S3P6,0001420529,None


In [618]:
# NYSE and NASDAQ symbols only (filter)
filtered_df = symbols_df[symbols_df['exchange'].isin(['XNYS', 'XNAS'])]

# select cols
filtered_df = filtered_df[['symbol', 'exchange', 'exchangeName']]

# filter df
print(filtered_df.head())
data_size = len(filtered_df.index)
print()
print(f"Data size: {data_size}") # get length of dataframe

  symbol exchange                 exchangeName
0      A     XNYS  New York Stock Exchange Inc
1     AA     XNYS  New York Stock Exchange Inc
4   AACG     XNAS           Nasdaq All Markets
5   AACI     XNAS           Nasdaq All Markets
6  AACIU     XNAS           Nasdaq All Markets

Data size: 7790


In [619]:
batch_size = 100
# Assuming filtered_df is a DataFrame that contains a 'symbol' column.
# Calculate total batches
total_batches = math.ceil(len(filtered_df) / batch_size)
print(f"Total batches: {total_batches}")

combined_data = []

# function -> fetch market cap data in batches
def fetch_market_cap(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=quote&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}
    
# Initialize a counter for completed batches
completed_batches = 0

# Define the length of the progress bar
progress_bar_length = 50

# batch processing - IMPORTANT (API efficiency)
for i in range(0, len(filtered_df), batch_size):
    batch_symbols = filtered_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_market_cap(batch_symbols, iex_key)  # Make sure iex_key is defined and valid
    
    # process and append data for each symbol in the batch
    for symbol in batch_symbols:
        market_cap = batch_data.get(symbol, {}).get('quote', {}).get('marketCap', None)
        combined_data.append({'symbol': symbol, 'marketcap': market_cap})
    
    # Increment the completed batches counter
    completed_batches += 1

    # Calculate the progress
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    
    # Create the progress bar
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    
    # Print the progress bar with the percentage
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Ensure the next print happens on the next line
print()

# convert combined data into a DataFrame
screener_df = pd.DataFrame(combined_data)

# join with the filtered_df - this adds exchange and exchangeName data
screener_df = screener_df.merge(filtered_df[['symbol', 'exchange', 'exchangeName']], on='symbol', how='left')

Total batches: 78
Progress: |██████████████████████████████████████████████████| 100.00% Complete


In [620]:
print(screener_df.head())
print()
print(f'Length: {len(screener_df.index)}')

  symbol     marketcap exchange                 exchangeName
0      A  3.930835e+10     XNYS  New York Stock Exchange Inc
1     AA  4.763415e+09     XNYS  New York Stock Exchange Inc
2   AACG  4.209672e+07     XNAS           Nasdaq All Markets
3   AACI  8.880890e+07     XNAS           Nasdaq All Markets
4  AACIU  8.848655e+07     XNAS           Nasdaq All Markets

Length: 7790


In [621]:
# drop NAs
screener_df.dropna(inplace=True)

# determins the count of droped NAs
print(f'New length: {len(screener_df)}')
print()

screener_df.head(5)

New length: 7688



,symbol,marketcap,exchange,exchangeName
0,A,3.930835e+10,XNYS,New York Stock Exchange Inc
1,AA,4.763415e+09,XNYS,New York Stock Exchange Inc
2,AACG,4.209672e+07,XNAS,Nasdaq All Markets
3,AACI,8.880890e+07,XNAS,Nasdaq All Markets
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets


In [622]:
batch_size = 100
total_batches = math.ceil(len(screener_df) / batch_size)
print(f"Total batches: {total_batches}")

quote_data_list = []

def fetch_quote_data(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=quote&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}
    
# initialize a counter for completed batches
completed_batches = 0

# define the length of the progress bar
progress_bar_length = 50

"""scalable endpoint variable data request"""
for i in range(0, len(screener_df), batch_size):
    batch_symbols = screener_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_quote_data(batch_symbols, iex_key)
    
    # process and append data for each symbol in the batch
    for symbol in batch_symbols:
        quote_data = batch_data.get(symbol, {}).get('quote', {})
        quote_data_list.append({
            'symbol': symbol, # KEY
            'latestPrice': quote_data.get('latestPrice', None),
            'close': quote_data.get('close', None), 
            'previousClose': quote_data.get('previousClose', None),
            'extendedPrice': quote_data.get('extendedPrice', None), 
            'extendedChange': quote_data.get('extendedChange', None), 
            'extendedChangePercent': quote_data.get('extendedChangePercent', None), 
            'latestVolume': quote_data.get('latestVolume', None), 
            'volume': quote_data.get('volume', None), 
            'previousVolume': quote_data.get('previousVolume', None), 
            'primaryExchange': quote_data.get('primaryExchange', None),
            'avgTotalVolume': quote_data.get('avgTotalVolume', None),
            'calculationPrice': quote_data.get('calculationPrice', None),
            'change': quote_data.get('change', None),
            'changePercent': quote_data.get('changePercent', None),
            'companyName': quote_data.get('companyName', None),
            # IEX real time prices - after hours without UTP authorization (maybe)
            'iexClose': quote_data.get('iexClose', None),
            'iexCloseTime': quote_data.get('iexCloseTime', None),
            'iexRealtimePrice': quote_data.get('iexRealtimePrice', None),
            'iexLastUpdated': quote_data.get('iexLastUpdated', None),
            'iexVolume': quote_data.get('iexVolume', None),
            # updates overkill
            'latestTime': quote_data.get('latestTime', None),
            'latestUpdate': quote_data.get('latestUpdate', None),
            # add parameters 'isUSMarketOpen', 'closeSource', 'openSource', 'iexOpen'
            'isUSMarketOpen': quote_data.get('isUSMarketOpen', None),
            'closeSource': quote_data.get('closeSource', None),
            'openSource': quote_data.get('openSource', None),
            'iexOpen': quote_data.get('iexOpen', None)})
    
    # increment completed batches counter
    completed_batches += 1

    # calculate progress
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    
    # create progress bar
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    
    # print progress bar with percentage
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Convert the combined data into a DataFrame
quote_df = pd.DataFrame(quote_data_list)

# MERGE DATA
screener_df = screener_df.merge(quote_df, on='symbol', how='left')

screener_df.head(5)

Total batches: 77


,symbol,marketcap,exchange,exchangeName,latestPrice,close,previousClose,extendedPrice,extendedChange,extendedChangePercent,latestVolume,volume,previousVolume,primaryExchange,avgTotalVolume,calculationPrice,change,changePercent,companyName,iexClose,iexCloseTime,iexRealtimePrice,iexLastUpdated,iexVolume,latestTime,latestUpdate,isUSMarketOpen,closeSource,openSource,iexOpen
0,A,3.930835e+10,XNYS,New York Stock Exchange Inc,134.1400,134.1400,134.84,134.14,0.0000,0.00000,1563909,1563909,1066838.0,NEW YORK STOCK EXCHANGE INC.,1311345,close,-0.7000,-0.00519,Agilent Technologies Inc.,134.14,1.708463e+12,134.14,1.708463e+12,43483.0,"February 20, 2024",1708462801959,False,official,official,134.07
1,AA,4.763415e+09,XNYS,New York Stock Exchange Inc,26.6900,26.6900,27.40,26.73,0.0400,0.00150,4711849,4711849,4686221.0,NEW YORK STOCK EXCHANGE INC.,5761875,close,-0.7100,-0.02591,Alcoa Corp,26.70,1.708463e+12,26.70,1.708463e+12,153167.0,"February 20, 2024",1708462875282,False,official,official,26.77
2,AACG,4.209672e+07,XNAS,Nasdaq All Markets,1.3406,1.3406,1.46,1.36,0.0194,0.01447,9245,9245,39354.0,NASDAQ,20623,close,-0.1194,-0.08178,ATA Creativity Global - ADR,1.46,1.708117e+12,0.00,0.000000e+00,0.0,"February 20, 2024",1708462800000,False,official,official,NaN
3,AACI,8.880890e+07,XNAS,Nasdaq All Markets,11.0200,NaN,10.98,11.02,0.0000,0.00000,33,33,502.0,NASDAQ,14409,close,0.0400,0.00364,Armada Acquisition Corp I,10.93,1.707235e+12,0.00,0.000000e+00,0.0,"February 20, 2024",1708462800000,False,official,official,NaN
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets,10.9000,NaN,10.89,10.90,0.0000,0.00000,0,0,12310.0,NASDAQ,6983,close,0.0100,0.00092,Armada Acquisition Corp I - Units (1 Ord & 1/2...,10.90,1.706906e+12,0.00,0.000000e+00,0.0,"February 2, 2024",1706907600000,False,official,official,NaN


In [623]:
def fetch_shares_outstanding_data(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=stats&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}

# Assuming screener_df is a DataFrame containing the symbols
batch_size = 100
total_batches = math.ceil(len(screener_df) / batch_size)
print(f"Total batches: {total_batches}")

shares_outstanding_data_list = []

# initialize a counter for completed batches
completed_batches = 0
# define the length of the progress bar
progress_bar_length = 50

# Replace 'screener_df' with the actual DataFrame containing the symbols
for i in range(0, len(screener_df), batch_size):
    batch_symbols = screener_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_shares_outstanding_data(batch_symbols, iex_key)
    
    # Process and append data for each symbol in the batch
    for symbol in batch_symbols:
        stats_data = batch_data.get(symbol, {}).get('stats', {})
        shares_outstanding_data_list.append({
            'symbol': symbol,
            'sharesOutstanding': stats_data.get('sharesOutstanding', None),
            'avg10Volume': stats_data.get('avg10Volume', None),
            'avg30Volume': stats_data.get('avg30Volume', None),
        })

    # increment completed batches counter
    completed_batches += 1

    # calculate progress
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    
    # create progress bar
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    
    # print progress bar with percentage
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Convert the combined data into a DataFrame
shares_outstanding_df = pd.DataFrame(shares_outstanding_data_list)

# MERGE DATA
screener_df = screener_df.merge(shares_outstanding_df, on='symbol', how='left')

screener_df.head(5)

Total batches: 77


,symbol,marketcap,exchange,exchangeName,latestPrice,close,previousClose,extendedPrice,extendedChange,extendedChangePercent,latestVolume,volume,previousVolume,primaryExchange,avgTotalVolume,calculationPrice,change,changePercent,companyName,iexClose,iexCloseTime,iexRealtimePrice,iexLastUpdated,iexVolume,latestTime,latestUpdate,isUSMarketOpen,closeSource,openSource,iexOpen,sharesOutstanding,avg10Volume,avg30Volume
0,A,3.930835e+10,XNYS,New York Stock Exchange Inc,134.1400,134.1400,134.84,134.14,0.0000,0.00000,1563909,1563909,1066838.0,NEW YORK STOCK EXCHANGE INC.,1311345,close,-0.7000,-0.00519,Agilent Technologies Inc.,134.14,1.708463e+12,134.14,1.708463e+12,43483.0,"February 20, 2024",1708462801959,False,official,official,134.07,293039707,1249900,1311345
1,AA,4.763415e+09,XNYS,New York Stock Exchange Inc,26.6900,26.6900,27.40,26.73,0.0400,0.00150,4711849,4711849,4686221.0,NEW YORK STOCK EXCHANGE INC.,5761875,close,-0.7100,-0.02591,Alcoa Corp,26.70,1.708463e+12,26.70,1.708463e+12,153167.0,"February 20, 2024",1708462875282,False,official,official,26.77,178471908,5162225,5761875
2,AACG,4.209672e+07,XNAS,Nasdaq All Markets,1.3406,1.3406,1.46,1.36,0.0194,0.01447,9245,9245,39354.0,NASDAQ,20623,close,-0.1194,-0.08178,ATA Creativity Global - ADR,1.46,1.708117e+12,0.00,0.000000e+00,0.0,"February 20, 2024",1708462800000,False,official,official,NaN,31401405,42101,20623
3,AACI,8.880890e+07,XNAS,Nasdaq All Markets,11.0200,NaN,10.98,11.02,0.0000,0.00000,33,33,502.0,NASDAQ,14409,close,0.0400,0.00364,Armada Acquisition Corp I,10.93,1.707235e+12,0.00,0.000000e+00,0.0,"February 20, 2024",1708462800000,False,official,official,NaN,8058884,461,14409
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets,10.9000,NaN,10.89,10.90,0.0000,0.00000,0,0,12310.0,NASDAQ,6983,close,0.0100,0.00092,Armada Acquisition Corp I - Units (1 Ord & 1/2...,10.90,1.706906e+12,0.00,0.000000e+00,0.0,"February 2, 2024",1706907600000,False,official,official,NaN,15000000,0,6983


In [635]:
"""THINK ABOUT MOVING THIS UP TO MAKE THE API CALLS MORE EFFICIENT"""
# print starting length
print(f'Length: {len(screener_df)}')
# filter out all negative changes from 'change'column
screener_df = screener_df[screener_df['change'] > 0]
# print new length after dropping the negative price movements
print(f'New length: {len(screener_df)}')
screener_df.head(5)

Length: 2801
New length: 2801


,symbol,marketcap,exchange,exchangeName,latestPrice,close,previousClose,extendedPrice,extendedChange,extendedChangePercent,latestVolume,volume,previousVolume,primaryExchange,avgTotalVolume,calculationPrice,change,changePercent,companyName,iexClose,iexCloseTime,iexRealtimePrice,iexLastUpdated,iexVolume,latestTime,latestUpdate,isUSMarketOpen,closeSource,openSource,iexOpen,sharesOutstanding,avg10Volume,avg30Volume,marketcapType
3,AACI,88808902.0,XNAS,Nasdaq All Markets,11.020,NaN,10.98,11.020,0.00,0.00000,33,33,502.0,NASDAQ,14409,close,0.040,0.00364,Armada Acquisition Corp I,10.930,1.707235e+12,0.00,0.000000e+00,0.0,"February 20, 2024",1708462800000,False,official,official,NaN,8058884,461,14409,Micro-Cap
4,AACIU,88486549.0,XNAS,Nasdaq All Markets,10.900,NaN,10.89,10.900,0.00,0.00000,0,0,12310.0,NASDAQ,6983,close,0.010,0.00092,Armada Acquisition Corp I - Units (1 Ord & 1/2...,10.900,1.706906e+12,0.00,0.000000e+00,0.0,"February 2, 2024",1706907600000,False,official,official,NaN,15000000,0,6983,Micro-Cap
5,AACT,527000000.0,XNYS,New York Stock Exchange Inc,10.540,10.54,10.52,10.540,0.00,0.00000,307792,307792,143900.0,NEW YORK STOCK EXCHANGE INC.,214072,close,0.020,0.00190,Ares Acquisition Corporation II - Class A,10.525,1.708108e+12,0.00,0.000000e+00,0.0,"February 20, 2024",1708462800000,False,official,official,NaN,50000000,140291,214072,Small-Cap
6,AACT=,526000002.0,XNYS,New York Stock Exchange Inc,10.629,NaN,10.60,10.629,0.00,0.00000,0,0,728.0,NEW YORK STOCK EXCHANGE INC.,4678,close,0.029,0.00274,Ares Acquisition Corporation II - Units (1 Ord...,10.570,1.706714e+12,0.00,0.000000e+00,0.0,"February 15, 2024",1708030800000,False,official,official,NaN,50000000,600,4678,Small-Cap
7,AADI,46844393.0,XNAS,Nasdaq All Markets,1.910,1.91,1.90,1.960,0.05,0.02618,178392,178392,184634.0,NASDAQ,227696,close,0.010,0.00526,Aadi Bioscience Inc,1.920,1.708463e+12,1.92,1.708463e+12,2735.0,"February 20, 2024",1708462800381,False,official,official,1.89,24525860,241834,227696,Micro-Cap


In [625]:
# categorize market cap
def categorize_market_cap(x):
    if x < 300e6:  #  < 300 million
        return 'Micro-Cap'
    elif 300e6 <= x < 2e9:  # 300 million to 2 billion
        return 'Small-Cap'
    elif 2e9 <= x < 10e9:  # 2 billion to 10 billion
        return 'Mid-Cap'
    elif 10e9 <= x < 200e9:  # 10 billion to 200 billion
        return 'Large-Cap'
    elif x >= 200e9:  # >= 200 billion
        return 'Mega-Cap'
    else:
        return 'Unknown'  # handles negative or NaN values

screener_df.loc[:, 'marketcapType'] = screener_df['marketcap'].apply(categorize_market_cap)

screener_df.head(10)

,symbol,marketcap,exchange,exchangeName,latestPrice,close,previousClose,extendedPrice,extendedChange,extendedChangePercent,latestVolume,volume,previousVolume,primaryExchange,avgTotalVolume,calculationPrice,change,changePercent,companyName,iexClose,iexCloseTime,iexRealtimePrice,iexLastUpdated,iexVolume,latestTime,latestUpdate,isUSMarketOpen,closeSource,openSource,iexOpen,sharesOutstanding,avg10Volume,avg30Volume,marketcapType
3,AACI,8.880890e+07,XNAS,Nasdaq All Markets,11.020,NaN,10.98,11.020,0.00,0.00000,33,33,502.0,NASDAQ,14409,close,0.040,0.00364,Armada Acquisition Corp I,10.930,1.707235e+12,0.00,0.000000e+00,0.0,"February 20, 2024",1708462800000,False,official,official,NaN,8058884,461,14409,Micro-Cap
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets,10.900,NaN,10.89,10.900,0.00,0.00000,0,0,12310.0,NASDAQ,6983,close,0.010,0.00092,Armada Acquisition Corp I - Units (1 Ord & 1/2...,10.900,1.706906e+12,0.00,0.000000e+00,0.0,"February 2, 2024",1706907600000,False,official,official,NaN,15000000,0,6983,Micro-Cap
5,AACT,5.270000e+08,XNYS,New York Stock Exchange Inc,10.540,10.54,10.52,10.540,0.00,0.00000,307792,307792,143900.0,NEW YORK STOCK EXCHANGE INC.,214072,close,0.020,0.00190,Ares Acquisition Corporation II - Class A,10.525,1.708108e+12,0.00,0.000000e+00,0.0,"February 20, 2024",1708462800000,False,official,official,NaN,50000000,140291,214072,Small-Cap
6,AACT=,5.260000e+08,XNYS,New York Stock Exchange Inc,10.629,NaN,10.60,10.629,0.00,0.00000,0,0,728.0,NEW YORK STOCK EXCHANGE INC.,4678,close,0.029,0.00274,Ares Acquisition Corporation II - Units (1 Ord...,10.570,1.706714e+12,0.00,0.000000e+00,0.0,"February 15, 2024",1708030800000,False,official,official,NaN,50000000,600,4678,Small-Cap
7,AADI,4.684439e+07,XNAS,Nasdaq All Markets,1.910,1.91,1.90,1.960,0.05,0.02618,178392,178392,184634.0,NASDAQ,227696,close,0.010,0.00526,Aadi Bioscience Inc,1.920,1.708463e+12,1.92,1.708463e+12,2735.0,"February 20, 2024",1708462800381,False,official,official,1.89,24525860,241834,227696,Micro-Cap
10,AAL,9.607046e+09,XNAS,Nasdaq All Markets,14.700,14.70,14.64,14.690,-0.01,-0.00068,30253070,30253070,21563210.0,NASDAQ,36042798,close,0.060,0.00410,American Airlines Group Inc,14.690,1.708463e+12,14.69,1.708463e+12,370718.0,"February 20, 2024",1708462800024,False,official,official,14.50,653540550,27622550,36042798,Mid-Cap
11,AAME,5.120974e+07,XNAS,Nasdaq All Markets,2.510,2.51,2.42,2.510,0.00,0.00000,12449,12449,8093.0,NASDAQ,6411,close,0.090,0.03719,Atlantic American Corp.,2.540,1.708463e+12,2.54,1.708463e+12,175.0,"February 20, 2024",1708464600000,False,official,official,2.35,20402288,4536,6411,Micro-Cap
12,AAN,3.190641e+08,XNYS,New York Stock Exchange Inc,10.520,10.52,10.51,10.520,0.00,0.00000,155364,155364,150643.0,NEW YORK STOCK EXCHANGE INC.,217206,close,0.010,0.00095,Aarons Company Inc (The),10.510,1.708463e+12,10.51,1.708463e+12,3926.0,"February 20, 2024",1708462802176,False,official,official,10.36,30329287,200152,217206,Small-Cap
17,AAPD,3.655002e+07,XNAS,Nasdaq All Markets,21.500,21.50,21.40,21.480,-0.02,-0.00093,294141,294141,271692.0,NASDAQ,261517,close,0.100,0.00467,Direxion Shares ETF Trust - Direxion Daily AAP...,21.540,1.708462e+12,21.54,1.708462e+12,812.0,"February 20, 2024",1708462800057,False,official,official,21.62,1700001,271539,261517,Micro-Cap
21,AAXJ,2.399488e+09,XNAS,Nasdaq All Markets,65.920,65.92,65.86,65.920,0.00,0.00000,782373,782373,585980.0,NASDAQ,883111,close,0.060,0.00091,BlackRock Institutional Trust Company N.A. - i...,65.960,1.708463e+12,65.96,1.708463e+12,26511.0,"February 20, 2024",1708462800081,False,official,official,66.26,36400000,809509,883111,Mid-Cap


In [626]:
# # calculate iexChange and iexChangePercent - custom calculation
# screener_df['u_iexChange'] = screener_df['iexClose'] - screener_df['close']
# screener_df['u_iexChangePercent'] = (screener_df['iexChange'] / screener_df['close'])

In [627]:
# print columns for screener_df
print(screener_df.columns)

Index(['symbol', 'marketcap', 'exchange', 'exchangeName', 'latestPrice',
       'close', 'previousClose', 'extendedPrice', 'extendedChange',
       'extendedChangePercent', 'latestVolume', 'volume', 'previousVolume',
       'primaryExchange', 'avgTotalVolume', 'calculationPrice', 'change',
       'changePercent', 'companyName', 'iexClose', 'iexCloseTime',
       'iexRealtimePrice', 'iexLastUpdated', 'iexVolume', 'latestTime',
       'latestUpdate', 'isUSMarketOpen', 'closeSource', 'openSource',
       'iexOpen', 'sharesOutstanding', 'avg10Volume', 'avg30Volume',
       'marketcapType'],
      dtype='object')


RESET FROM HERE

In [646]:
# create a columns filter
screener_cols = ['symbol', # KEY
                 # market cap
                'marketcap',
                'marketcapType', 
                # price(s)
                'close', 
                'latestPrice', 
                'extendedPrice',
                # relative change
                'calculationPrice',
                'change', #mandatory
                'changePercent', 
                # shares (approx. float)
                'sharesOutstanding', 
                # volume
                'latestVolume',
                'avgTotalVolume', # NEW ############################ 
                'avg10Volume',
                'avg30Volume',
                'iexVolume', # NEW ############################
                #info
                'isUSMarketOpen',
                'exchange',
                'exchangeName',
                'companyName']
                # 'closeSource',
                # 'openSource',
                # IEX real time prices - after hours without UTP authorization
                # 'iexClose',
                # 'iexCloseTime',
                # 'iexRealtimePrice',
                # 'u_iexChange',
                # 'u_iexChangePercent',
                # 'iexOpen'] 
                #### ADD -  CHANGE % (CALCULATION - custom) ####
            

screener_df_x = screener_df[screener_cols]

screener_df_x.head(25)

,symbol,marketcap,marketcapType,close,latestPrice,extendedPrice,calculationPrice,change,changePercent,sharesOutstanding,latestVolume,avgTotalVolume,avg10Volume,avg30Volume,iexVolume,isUSMarketOpen,exchange,exchangeName,companyName
3,AACI,8.880890e+07,Micro-Cap,NaN,11.0200,11.0200,close,0.0400,0.00364,8058884,33,14409,461,14409,0.0,False,XNAS,Nasdaq All Markets,Armada Acquisition Corp I
4,AACIU,8.848655e+07,Micro-Cap,NaN,10.9000,10.9000,close,0.0100,0.00092,15000000,0,6983,0,6983,0.0,False,XNAS,Nasdaq All Markets,Armada Acquisition Corp I - Units (1 Ord & 1/2...
5,AACT,5.270000e+08,Small-Cap,10.5400,10.5400,10.5400,close,0.0200,0.00190,50000000,307792,214072,140291,214072,0.0,False,XNYS,New York Stock Exchange Inc,Ares Acquisition Corporation II - Class A
6,AACT=,5.260000e+08,Small-Cap,NaN,10.6290,10.6290,close,0.0290,0.00274,50000000,0,4678,600,4678,0.0,False,XNYS,New York Stock Exchange Inc,Ares Acquisition Corporation II - Units (1 Ord...
7,AADI,4.684439e+07,Micro-Cap,1.9100,1.9100,1.9600,close,0.0100,0.00526,24525860,178392,227696,241834,227696,2735.0,False,XNAS,Nasdaq All Markets,Aadi Bioscience Inc
10,AAL,9.607046e+09,Mid-Cap,14.7000,14.7000,14.6900,close,0.0600,0.00410,653540550,30253070,36042798,27622550,36042798,370718.0,False,XNAS,Nasdaq All Markets,American Airlines Group Inc
11,AAME,5.120974e+07,Micro-Cap,2.5100,2.5100,2.5100,close,0.0900,0.03719,20402288,12449,6411,4536,6411,175.0,False,XNAS,Nasdaq All Markets,Atlantic American Corp.
12,AAN,3.190641e+08,Small-Cap,10.5200,10.5200,10.5200,close,0.0100,0.00095,30329287,155364,217206,200152,217206,3926.0,False,XNYS,New York Stock Exchange Inc,Aarons Company Inc (The)
17,AAPD,3.655002e+07,Micro-Cap,21.5000,21.5000,21.4800,close,0.1000,0.00467,1700001,294141,261517,271539,261517,812.0,False,XNAS,Nasdaq All Markets,Direxion Shares ETF Trust - Direxion Daily AAP...
21,AAXJ,2.399488e+09,Mid-Cap,65.9200,65.9200,65.9200,close,0.0600,0.00091,36400000,782373,883111,809509,883111,26511.0,False,XNAS,Nasdaq All Markets,BlackRock Institutional Trust Company N.A. - i...


In [647]:
# drop all NaN when 'close' is NaN
print(f'Length: {len(screener_df_x)}')
screener_df_x = screener_df_x.dropna(subset=['close'])
print(f'New Length: {len(screener_df_x)}')

Length: 2801
New Length: 2556


In [648]:
# create five (5) dataframes for each marketcap type
microcap_df = screener_df_x[screener_df_x['marketcapType'] == 'Micro-Cap']
smallcap_df = screener_df_x[screener_df_x['marketcapType'] == 'Small-Cap']
midcap_df = screener_df_x[screener_df_x['marketcapType'] == 'Mid-Cap']
largecap_df = screener_df_x[screener_df_x['marketcapType'] == 'Large-Cap']
megacap_df = screener_df_x[screener_df_x['marketcapType'] == 'Mega-Cap']

In [639]:
# # print
# largecap_df.head(5)

,symbol,marketcap,marketcapType,close,latestPrice,extendedPrice,calculationPrice,change,changePercent,sharesOutstanding,latestVolume,avgTotalVolume,avg10Volume,avg30Volume,iexVolume,isUSMarketOpen,exchange,exchangeName,companyName
29,ABEV,4.032981e+10,Large-Cap,2.56,2.56,2.58,close,0.01,0.00392,15753833284,24204251,8719356,7504348,8719356,532248.0,False,XNYS,New York Stock Exchange Inc,Ambev S.A. - ADR
71,ACGLN,3.229315e+10,Large-Cap,20.27,20.27,20.18,close,0.10,0.00496,20000000,13430,28120,28479,28120,566.0,False,XNAS,Nasdaq All Markets,Arch Capital Group Ltd - 4.55% PRF PERPETUAL U...
115,ADI,9.391202e+10,Large-Cap,189.40,189.40,189.87,close,1.16,0.00616,495839582,4600510,3583926,4802108,3583926,138190.0,False,XNAS,Nasdaq All Markets,Analog Devices Inc.
143,AEE,1.837197e+10,Large-Cap,69.87,69.87,69.87,close,0.36,0.00518,262945048,1530441,2026789,2056887,2026789,53273.0,False,XNYS,New York Stock Exchange Inc,Ameren Corp.
145,AEG,1.045283e+10,Large-Cap,5.76,5.76,5.95,close,0.06,0.01053,1814726912,2251760,2070788,2315040,2070788,26612.0,False,XNYS,New York Stock Exchange Inc,Aegon Ltd. - New York Shares


In [640]:
# get columns
# print(largecap_df.columns)

Index(['symbol', 'marketcap', 'marketcapType', 'close', 'latestPrice',
       'extendedPrice', 'calculationPrice', 'change', 'changePercent',
       'sharesOutstanding', 'latestVolume', 'avgTotalVolume', 'avg10Volume',
       'avg30Volume', 'iexVolume', 'isUSMarketOpen', 'exchange',
       'exchangeName', 'companyName'],
      dtype='object')


In [641]:
# largecap_df['changePercent']

29      0.00392
71      0.00496
115     0.00616
143     0.00518
145     0.01053
         ...   
7587    0.01367
7606    0.00543
7634    0.00052
7640    0.00217
7674    0.02002
Name: changePercent, Length: 424, dtype: float64

In [649]:
def filter_stocks(df, 
                  market_cap_type=None, 
                  price_min=None, 
                  change_min_percent=None,  # Pre-market gap percentage
                  volume_min=None, 
                  volume_avg_comparison=None,  # '10day' or '30day'
                  shares_outstanding_min=None, 
                  shares_outstanding_max=None,
                  extended_price_min=None):  # For pre-market gap analysis
    # Market Cap Type
    if market_cap_type is not None:
        df = df[df['marketcapType'] == market_cap_type]

    # Change Percent
    if change_min_percent is not None:
        df = df[df['changePercent'] >= change_min_percent]
    
    # PRICE
    if price_min is not None:
        df = df[df['latestPrice'] >= price_min]
    
    # Ensure 'close' and 'extendedPrice' are not null to avoid division by zero or null comparisons
    df = df.dropna(subset=['close', 'extendedPrice'])
    
    # Filter for gap-ups: only include stocks where extendedPrice > close
    df = df[df['extendedPrice'] > df['close']]
    
    # VOLUME
    if volume_min is not None:
        df = df[df['latestVolume'] >= volume_min]
    
    # Dynamic Volume Comparison
    if volume_avg_comparison is not None:
        if volume_avg_comparison == '10day':
            df = df[df['latestVolume'] >= df['avg10Volume']]
        elif volume_avg_comparison == '30day':
            df = df[df['latestVolume'] >= df['avg30Volume']]
    
    # SHARES OUTSTANDING
    if shares_outstanding_min is not None:
        df = df[df['sharesOutstanding'] >= shares_outstanding_min]
    if shares_outstanding_max is not None:
        df = df[df['sharesOutstanding'] <= shares_outstanding_max]
    
    # EXTENDED PRICE (For pre-market gap analysis)
    if extended_price_min is not None:
        df = df[df['extendedPrice'] >= extended_price_min]
    
    return df


In [652]:
#### MICROCAP FILTER ####
microcap_df = filter_stocks(
    microcap_df, 
    market_cap_type='Micro-Cap', 
    price_min=0.50,  # Keep minimum price to filter out ultra-low-priced stocks
    change_min_percent=.03,  # Increase to target more significant gaps; adjusted from 0.02 to 3%
    volume_min=10000,  # New: Set a minimum volume threshold to ensure liquidity
    volume_avg_comparison='10day'  # Ensure current volume is above the 10-day average
)

# Order descending by changePercent to prioritize highest gap-ups
microcap_df = microcap_df.sort_values('changePercent', ascending=False)

# After sorting, re-index the dataframe for clarity
microcap_df = microcap_df.reset_index(drop=True)

# # get first 10 and inplace the dataframe
# microcap_df = microcap_df.iloc[:10] 

# Print the length of the dataframe to confirm the number of targeted stocks
print(f"Length: {len(microcap_df)}")

microcap_df.head(5)

Length: 55


,symbol,marketcap,marketcapType,close,latestPrice,extendedPrice,calculationPrice,change,changePercent,sharesOutstanding,latestVolume,avgTotalVolume,avg10Volume,avg30Volume,iexVolume,isUSMarketOpen,exchange,exchangeName,companyName
0,LUNR,282835991.0,Micro-Cap,10.99,10.99,12.09,close,3.67,0.50137,25735759,64341117,5468089,14401985,5468089,411144.0,False,XNAS,Nasdaq All Markets,Intuitive Machines Inc - Class A
1,MNY,73354528.0,Micro-Cap,3.06,3.06,3.10,close,0.96,0.45714,23972068,15003748,1136014,2171659,1136014,94840.0,False,XNAS,Nasdaq All Markets,MoneyHero Ltd - Class A
2,GRYP,91411187.0,Micro-Cap,2.36,2.36,2.50,close,0.47,0.24868,38733554,1223157,158437,500017,158437,6011.0,False,XNAS,Nasdaq All Markets,Gryphon Digital Mining Inc.
3,ICU,78274659.0,Micro-Cap,1.42,1.42,1.44,close,0.26,0.22414,55122999,15363948,7703670,13791161,7703670,131347.0,False,XNAS,Nasdaq All Markets,SeaStar Medical Holding Corp
4,VTYX,176553843.0,Micro-Cap,2.99,2.99,3.06,close,0.53,0.21545,59048108,9639347,1285441,1471845,1285441,168273.0,False,XNAS,Nasdaq All Markets,Ventyx Biosciences Inc


In [ ]:
#### SMALL-CAP FILTER ####

In [ ]:
#### MID-CAP FILTER ####

In [653]:
#### LARGE-CAP FILTER ####
largecap_df = filter_stocks(
    df=largecap_df,  # Your DataFrame of Large-Cap stocks
    market_cap_type='Large-Cap',
    price_min=10,  # Adjust as needed for large-cap stocks
    change_min_percent=.01,  # Looking for at least a 0.5% pre-market gap up
    volume_min=50000,  # Setting a volume minimum to ensure liquidity
    volume_avg_comparison='10day'  # Comparing to 10-day average volume to gauge interest
)


# Order descending by changePercent to prioritize highest gap-ups
largecap_df = largecap_df.sort_values('changePercent', ascending=False)

# After sorting, re-index the dataframe for clarity
largecap_df = largecap_df.reset_index(drop=True)

# # get first 10 and inplace the dataframe
# largecap_df = largecap_df.iloc[:10] 

# Print the length of the dataframe to confirm the number of targeted stocks
print(f"Length: {len(largecap_df)}")

largecap_df.head(5)

Length: 24


,symbol,marketcap,marketcapType,close,latestPrice,extendedPrice,calculationPrice,change,changePercent,sharesOutstanding,latestVolume,avgTotalVolume,avg10Volume,avg30Volume,iexVolume,isUSMarketOpen,exchange,exchangeName,companyName
0,DFS,3.093717e+10,Large-Cap,123.72,123.72,124.05,close,13.8516,0.12607,250057955,22451052,1851805,1508909,1851805,991862.0,False,XNYS,New York Stock Exchange Inc,Discover Financial Services
1,CAG,1.361358e+10,Large-Cap,28.48,28.48,28.50,close,1.2700,0.04667,478005028,5821383,4715532,3789816,4715532,188064.0,False,XNYS,New York Stock Exchange Inc,Conagra Brands Inc
2,BBWI,1.070055e+10,Large-Cap,47.36,47.36,47.40,close,1.9600,0.04317,225940592,4162398,2238327,2052499,2238327,150435.0,False,XNYS,New York Stock Exchange Inc,Bath & Body Works Inc
3,EXAS,1.134653e+10,Large-Cap,62.74,62.74,62.75,close,1.7700,0.02903,180849944,2168495,1791026,2045031,1791026,87831.0,False,XNAS,Nasdaq All Markets,Exact Sciences Corp.
4,INTC,1.882306e+11,Large-Cap,44.52,44.52,44.58,close,1.0100,0.02321,4228000000,61514136,48406494,39583125,48406494,770614.0,False,XNAS,Nasdaq All Markets,Intel Corp.


In [ ]:
#### MEGA-CAP FILTER ####

In [ ]:
# ADD - relative volume (ratio) data???
# some of this you might have already covered, but might be able to get more granular with the data

In [ ]:
# create daily watchlist dataframe
# combine the filtered dataframes

In [ ]:
# news

In [ ]:
# news international

In [ ]:
# sentiment - custom

In [ ]:
# RBV - robust value score - custom

In [ ]:
# key levels from historical data
# other data

In [ ]:
# exogenous data

In [ ]:
# do work with BENZINGA BZ squawk box - see if there is an API

In [ ]:
# RSI, MACD, etc. - custom

In [ ]:
# ichimoku cloud - custom

END - END - END - IN PROGRESS - END - END - END END - END - END - IN PROGRESS - END - END - END END - END - END - IN PROGRESS - END - END - END


In [ ]:
# YAHOO data - testing

# data feed testing
import yfinance as yf
import pandas as pd

# List of stock symbols
symbols = ['PLTR', 'TSLA','NVDA']

# Fetch data
data = []
for symbol in symbols:
    stock = yf.Ticker(symbol)
    hist = stock.history(period="1d")  # Get the last day's data
    info = stock.info  # Get general stock info
    
    # Extracting the required information
    data.append({
        'Symbol': symbol,
        'Close Price': hist['Close'].iloc[-1] if not hist.empty else None,
        'Volume': hist['Volume'].iloc[-1] if not hist.empty else None,
        'Average Volume': info.get('averageVolume'),
        # get after hours current price
        'After Hours Price': info.get('postMarketPrice'),
        'Change in Price': info.get('regularMarketChange'),
        'Change in Percentage': info.get('regularMarketChangePercent'),
    })

# Convert to DataFrame
df = pd.DataFrame(data)

print(df)


  Symbol  Close Price     Volume  Average Volume After Hours Price  \
0   PLTR    23.400000   93067100        69071645              None   
1   TSLA   193.759995  104332300       111586877              None   
2   NVDA   694.520020   70171600        44411586              None   

  Change in Price Change in Percentage  
0            None                 None  
1            None                 None  
2            None                 None  


In [ ]:
"""
IMPORTANT: Need UTP autorization to continue
https://iexcloud.io/documentation/using-core-data/getting-nasdaq-listed-utp-otc-stock-data.html
Step 1: https://www.utpplan.com/DOC/VendorAgreement.pdf
Step 2: https://www.utpplan.com/datafeed_approval
Step 3: https://www.utpplan.com/system_application

"""

# create a dataframe from utp_auth_columns.xlsx
utp_auth_df = pd.read_excel('utp_auth_columns.xlsx')
# replace NA with " " 
utp_auth_df = utp_auth_df.fillna(" ")

utp_auth_df

,utp_auth_required,humbled_trader_TARGETs,iex_real-time_alternatives,u_screener_df_columns_ACTIVE
0,close,Symbol,,
1,closeTime,Price ($),,
2,delayedPrice,Float (shares),,
3,delayedPriceTime,Change Close (%),,
4,extendedPrice,Volume Today (shares),,
5,extendedPriceTime,Avereage Volume (5day) (shares/day),,
6,extendedChange,Market Cap ($),,
7,extendedChangePercent,Held By Institutions (%),,
8,high,Sector,,
9,low,Company Name,,


In [ ]:
# # DEV - marketcap groupby count
# marketcap_counts = screener_df['marketcapType'].value_counts()

# # convert SERIES to DF
# marketcap_counts_df = marketcap_counts.reset_index()
# marketcap_counts_df.columns = ['marketcapType', 'count']

# # add percentage column
# total_count = marketcap_counts.sum()
# marketcap_counts_df['percentage'] = (marketcap_counts_df['count'] / total_count) * 100

# # print
# marketcap_counts_df

In [ ]:
# # DEV - checker
# total_count_CHECK = marketcap_counts_df['count'].sum()

# # if statement comparing the two values, return 'Data is correct' if they match, otherwise return 'Data is incorrect'
# if total_count_CHECK == len(screener_df):
#     print('Data lengths match! SUCCESS')
# else:    
#     print('Data mismatch! ERROR')
#     print()
#     print(f'Code checker: {total_count_CHECK}')
#     print(f'Data length: {len(screener_df)}')

In [ ]:
# def print_available_stats_parameters(symbol, iex_key):
#     url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={iex_key}'
#     response = requests.get(url)
#     if response.status_code == 200:
#         data = response.json()
#         print(json.dumps(data, indent=4, sort_keys=True))
#     else:
#         print(f"Failed to fetch stats for {symbol}. Status code: {response.status_code}")

# pltr = 'PLTR'
# print_available_stats_parameters(pltr, iex_key)

In [ ]:
# def fetch_deep_trades_data(symbols_batch, iex_key):
#     symbols_str = ','.join(symbols_batch)
#     url = f'https://cloud.iexapis.com/stable/deep/trades?symbols={symbols_str}&token={iex_key}'
#     response = requests.get(url)
#     if response.status_code == 200:
#         return response.json()
#     else:
#         return {}

# # Assuming screener_df is a DataFrame containing the symbols
# batch_size = 100
# # Example for illustrative purposes
# # screener_df = pd.DataFrame({'symbol': ['AAPL', 'MSFT', 'GOOGL']})
# total_batches = math.ceil(len(screener_df) / batch_size)
# print(f"Total batches: {total_batches}")

# deep_data_list = []

# for i in range(0, len(screener_df), batch_size):
#     batch_symbols = screener_df['symbol'].iloc[i:i+batch_size].tolist()
#     batch_data = fetch_deep_trades_data(batch_symbols, iex_key)
    
#     # Process and infer volume data for each symbol in the batch
#     for symbol in batch_symbols:
#         if symbol in batch_data and batch_data[symbol]:
#             volume = sum(trade['size'] for trade in batch_data[symbol])
#             deep_data_list.append({
#                 'symbol': symbol,
#                 'volume': volume,
#             })

# # Convert the combined data into a DataFrame
# iex_deep_df = pd.DataFrame(deep_data_list)

# print(iex_deep_df.head(20)) # data check
# print(len(iex_deep_df.index)) # data check

In [ ]:
# print(f"screener_df column count: {len(screener_df.columns)}") # col count
# screener_df = screener_df.merge(shares_outstanding_df, on='symbol', how='left')

In [ ]:
# # My Watchlist
# watchlist = ['PLTR', 'TSLA', 'NOW', 'SNOW','FB', 'NVDA', 'PYPL', 'ADBE', 'NFLX']
# watchlist_df = screener_df[screener_df['symbol'].isin(watchlist)]

# watchlist_df

In [ ]:
# # DEV - API endpoing parameter availability. 
# schema_base_test_url = 'https://cloud.iexapis.com/stable'
# test_symbol = 'PLTR' # test ticker symbol
# schema_test_url = f"{schema_base_test_url}/stock/{test_symbol}/quote?schema=true&token={iex_key}"
# # GET request
# schema_test_response = requests.get(schema_test_url)

# print(json.dumps(schema_test_response.json(), indent=4))


IDEAS
1. Robust value score (RBV)
2. create data frames for each market cap
3. create a function that does math and querying based on rules / conditions
4. apply function to the market cap data sets
5. Look at other STATS now that the API is connected for the sharesOutstanding

NOTES
1. Float
1. Free Flt Mkt Cap
1. Free Flt
1. Shares outstanding

MANDATORY FIELDS
1. 